In [ ]:
#-*- coding: utf-8 -*-
# 라이브러리 import

try:
    from selenium import webdriver
except:
    !pip install selenium
    from selenium import webdriver

try:
    from webdriver_manager.chrome import ChromeDriverManager
except:
    !pip install webdriver_manager
    from webdriver_manager.chrome import ChromeDriverManager

import datetime
from dateutil.relativedelta import relativedelta
try:
    from urllib.parse import urlencode
except:
    !pip install urllib3
    from urllib.parse import urlencode

import time
import calendar

try:
    import csv
except:
    !pip install csv
    import csv

import re

In [ ]:
# 브라우저 옵션
options = webdriver.ChromeOptions()
options.add_argument('start-maximized')
options.add_argument(
            "user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36"
        )
options.add_argument("lang=ko_KR")

In [ ]:
# 문자열 리스트에 문자 존재 반환 함수
def listIndexOf(list, str):
    list_index = 1
    try:
        list_index = list.index(str)
    except ValueError:
        list_index = -1
    return list_index

In [ ]:
# 루프 제한 값
sleepMaxCount = 5
# select query
item_query = "section.css-1dbjc4n > div.css-1dbjc4n > div > div"
link_query = "a.a.css-4rbku5.css-18t94o4.css-901oao.r-m0bqgq.r-1loqt21.r-1q142lx.r-1qd0xha.r-a023e6.r-16dba41.r-rjixqe.r-bcqeeo.r-3s2u2q.r-qvutc0"
time_query = "a.css-4rbku5.css-18t94o4.css-901oao.r-m0bqgq.r-1loqt21.r-1q142lx.r-1qd0xha.r-a023e6.r-16dba41.r-rjixqe.r-bcqeeo.r-3s2u2q.r-qvutc0 > time"
name_query = "a.css-4rbku5.css-18t94o4.css-1dbjc4n.r-1loqt21.r-1wbh5a2.r-dnmrzs.r-1ny4l3l > div > div > div > span > span"
id_query = "a.css-4rbku5.css-18t94o4.css-1dbjc4n.r-1loqt21.r-1wbh5a2.r-dnmrzs.r-1ny4l3l > div > div.css-1dbjc4n.r-18u37iz.r-1wbh5a2.r-13hce6t > div > span"
contents_query = "div.css-1dbjc4n > div.css-901oao.r-18jsvk2.r-1qd0xha.r-a023e6.r-16dba41.r-rjixqe.r-bcqeeo.r-bnwqim.r-qvutc0"

In [ ]:
# 데이터 추출 함수
def searchWeb(chrome, cvsFileName, search_encode, start_date, end_date, encode_type) :
    chrome.get("https://twitter.com/search?f=live&" + search_encode + "%20until%3A" + end_date + "%20since%3A" + start_date + "&src=typed_query")
    item_query = "section.css-1dbjc4n > div.css-1dbjc4n > div > div"

    i=0
    dataRow=0
    isEnd=0
    time.sleep(5)
    item_len = 0
    link_hrefs = []
    contants_txts = []
    new_items = []
    while item_len==0 and isEnd < sleepMaxCount:
        time.sleep(1)
        items = chrome.find_elements_by_css_selector(item_query)
        item_len = items.__len__()
        isEnd = isEnd + 1
    
    number = 0
    
    while True:
        print("스크롤 : ", i)
        # 중복 되지 않은 데이터만 new_items에 담는다.
        new_items = []
        items = chrome.find_elements_by_css_selector(item_query)
        for item in items:
            try:
                href = item.find_element_by_css_selector(link_query).get_attribute("href")
                if listIndexOf(link_hrefs, href) == -1:
                    new_items.append(item)
                    link_hrefs.append(href)
            except Exception as e:
                print("no search link")
                try:
                    contents_txt = item.find_element_by_css_selector(contents_query).text
                    if listIndexOf(contants_txts, contents_txt) == -1:
                        new_items.append(item)
                        contants_txts.append(contents_txt)
                except Exception as ex:
                    print("no search contents")

        if len(new_items) == 0 :
            break
            
        for item in new_items:
            with open(cvsFileName, "a", newline="", encoding=encode_type) as f:
                w = csv.writer(f)
                number=number+1
                time_txt = ""
                name_txt = ""
                id_txt = ""
                contents_txt = ""
                contents_urls = ""

                try:
                    time_txt = item.find_element_by_css_selector(time_query).get_attribute("datetime")
                except Exception as e:
                    print("no search time")
                    print(e)

                try:
                    name_txt = item.find_element_by_css_selector(name_query).text
                except Exception as e:
                    print("no search name")

                try:
                    id_txt = item.find_element_by_css_selector(id_query).text
                except Exception as e:
                    print("no search id")

                try:
                    contents_txt = item.find_element_by_css_selector(contents_query).text
                except Exception as e:
                    print("no search contents")

                contents_url_list = re.findall("https?://[^\s]+", contents_txt)
                if len(contents_url_list) > 0:
                    for contents_url in contents_url_list:
                        if contents_url.find("https://twitter.com") == -1:
                            if contents_urls != "":
                                contents_urls += ","
                            contents_urls += contents_url

                w.writerow([number, time_txt, name_txt, id_txt, contents_txt, contents_urls])
                print(number, " => name : ", name_txt, ", id : ", id_txt)
               
        
        # 스크롤 맨 아래로 이동
        chrome.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        i = i+1
        print("스크롤 : ", i)
        time.sleep(5)



In [ ]:
# 날짜 혹은 월별로 나누어서 진행 하기위한 함수
def startSearch() :
    search_txt = input("search : ")
    date_type = input("search date type (m=월, d=일, default: m) : ")

    now_str = datetime.datetime.now().strftime("%Y-%m-%d")
    now_str_arr = now_str.split("-")
    now_time = datetime.date(int(now_str_arr[0]), int(now_str_arr[1]), int(now_str_arr[2]))

    dsdate_str = now_time.strftime("%Y-%m-%d")
    dedate_str = now_time.strftime("%Y-%m-%d")
    if date_type != "d":
        dsdate_str = now_time.strftime("%Y-%m")
        dedate_str = now_time.strftime("%Y-%m")

    input_sdate = input("start date ("+dsdate_str+") : ")
    input_edate = input("end date ("+dedate_str+") : ")

    if input_sdate is None or input_sdate == "":
        input_sdate = dsdate_str

    if input_edate is None or input_edate == "":
        input_edate = dedate_str

    start_time = now_time
    end_time = now_time

    if date_type != "d":
        sdate_arr = input_sdate.split("-")
        edate_arr = input_edate.split("-")
        start_time = datetime.date(int(sdate_arr[0]), int(sdate_arr[1]), 1)
        end_time = datetime.date(int(edate_arr[0]), int(edate_arr[1]), 1) + relativedelta(months=1) - datetime.timedelta(days=1)
    else:
        sdate_arr = input_sdate.split("-")
        edate_arr = input_edate.split("-")
        start_time = datetime.date(int(sdate_arr[0]), int(sdate_arr[1]), int(sdate_arr[2]))
        end_time = datetime.date(int(edate_arr[0]), int(edate_arr[1]), int(edate_arr[2]))

    if end_time > now_time:
        end_time = now_time

    sq = {"q": search_txt}
    txt_encode = urlencode(sq)

    run_time = end_time
    encode_type="utf-8"

    # 초기 파일 생성
    cvsFileName = "twitter_search_" + search_txt + "_" + input_sdate + "_" + input_edate + ".csv"
    with open(cvsFileName, "w", newline="", encoding=encode_type) as f:
        colnames = ['number', 'time', 'name', 'id', 'contents', 'content_urls']
        w = csv.writer(f)
        w.writerow(colnames)

    print("start_time: ", start_time.strftime("%Y-%m-%d"))
    print("end_time: ", end_time.strftime("%Y-%m-%d"))

    chrome = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    try:
        while_loop = True
        while while_loop :
            if date_type != "d": # 월단위 검색
                run_time_str = run_time.strftime("%Y-%m")
                print("검색 월 : " + run_time_str)
                run_date_sp = run_time_str.split("-")
                run_start_time = datetime.datetime(int(run_date_sp[0]), int(run_date_sp[1]), 1)
                run_end_time = datetime.date(int(run_date_sp[0]), int(run_date_sp[1]), 1) + relativedelta(months=1) - datetime.timedelta(days=1)

                print(chrome, cvsFileName, txt_encode, run_start_time.strftime("%Y-%m-%d"), run_end_time.strftime("%Y-%m-%d"), encode_type)
                time.sleep(1)
                searchWeb(chrome, cvsFileName, txt_encode, run_start_time.strftime("%Y-%m-%d"), run_end_time.strftime("%Y-%m-%d"), encode_type)
                run_time = datetime.date(int(run_date_sp[0]), int(run_date_sp[1]), 1) - relativedelta(months=1)

                if start_time > run_time:
                    while_loop = False

            else:
                print("검색 날짜 : " + run_time.strftime("%Y-%m-%d"))
                run_end_time = run_time + datetime.timedelta(days=1)

                print(chrome, cvsFileName, txt_encode, run_time.strftime("%Y-%m-%d"), run_end_time.strftime("%Y-%m-%d"), encode_type)
                time.sleep(1)
                searchWeb(chrome, cvsFileName, txt_encode, run_time.strftime("%Y-%m-%d"), run_end_time.strftime("%Y-%m-%d"), encode_type)
                run_time = run_time - datetime.timedelta(days=1)
                if start_time > run_time:
                    while_loop = False
    except Exception as e:
        print(e)
    finally:
        if chrome :
            chrome.close()


In [ ]:
startSearch()